# EXPLORATION OF SPARSE MATRIX

# IMPORTS

In [35]:
import numpy as np
from itertools import product
from typing import *
from scipy.sparse import csr_matrix, csc_matrix
import scipy
import pathlib

# CREATION OF MATRIX WITH NUMPY

In [1]:
"""
The create_matrix function generates a NumPy array (matrix) with n_rows rows and n_cols columns, and a specified number of non-zero elements (n_nonzero).

It first initializes a matrix of all zeros using the np.zeros function with the specified number of rows and columns, and a data type of 32-bit integer (dtype = np.int32).

It then enters a loop that generates n_nonzero random non-zero values in the matrix. In each iteration of the loop, it selects a random row and column index using the np.random.randint function, and assigns a random integer between 1 and 1000 (exclusive of 1000) to the corresponding element in the matrix.

After generating the non-zero elements, it prints the number of non-zero elements in the matrix, and the total number of elements (i.e. the product of n_rows and n_cols) for informational purposes.

Finally, it returns the generated matrix.
"""

def create_matrix(n_rows, n_cols, n_nonzero):
    matrix = np.zeros((n_rows, n_cols), dtype = np.int32)
    for i in range(n_nonzero):
        row = np.random.randint(n_rows)
        col = np.random.randint(n_cols)
        matrix[row, col] = np.random.randint(1, 1_000)
    print(f"{np.count_nonzero(matrix):,}")
    print(f"{n_rows*n_cols:,}")
    return matrix

# MATRIX MULTIPLICATION PYTHON IMPELEMTNATIONS

In [29]:
"""
basic matrix multiplication following matrix multiplication formula
res[i][j] = sum(mat1[i][k] * mat2[k][j]) where k in is the number of columns for mat1, and number of rows for mat2
"""
class Solution1:
    def multiply(self, mat1: List[List[int]], mat2: List[List[int]]) -> List[List[int]]:
        m, n, k = len(mat1), len(mat2[0]), len(mat2)
        M = [[0]*n for _ in range(m)]
        for i in range(m):
            for j in range(n):
                M[i][j] = sum(mat1[i][ii]*mat2[ii][j] for ii in range(k))
        return M

In [30]:
"""
matrix multiplication that skips 0s
"""
class Solution2:
    def multiply(self, mat1: List[List[int]], mat2: List[List[int]]) -> List[List[int]]:
        m, n, k = len(mat1), len(mat2[0]), len(mat2)
        M = [[0]*n for _ in range(m)]
        for i in range(m):
            for ii in range(k):
                if mat1[i][ii] == 0: continue
                for j in range(n):
                    M[i][j] += mat1[i][ii]*mat2[ii][j]
        return M

In [31]:
"""
This code defines two classes, CompressedSparseRowMatrix and CompressedSparseColumnMatrix, that represent a matrix in compressed sparse row (CSR) and compressed sparse column (CSC) formats, respectively.

Both classes take a two-dimensional list (matrix) as input to their constructors. The classes then store the non-zero values of the matrix in a list (values) and their corresponding row and column indices in separate lists (row_indices for CSR and col_indices for CSC).

The Solution3 class defines a multiply method that takes two matrices (mat1 and mat2) as input and returns their matrix product in the form of a two-dimensional list. The method first initializes the dimensions of the matrices and creates objects of the CompressedSparseRowMatrix and CompressedSparseColumnMatrix classes for the left and right matrices, respectively.

It then initializes the result matrix (res) as a two-dimensional list of zeros with dimensions R by C. The method then loops over each row and column index of the result matrix using the product function from the itertools module.

For each element in the result matrix, it performs a sparse matrix multiplication by iterating over the non-zero elements in the corresponding row of the left matrix and the non-zero elements in the corresponding column of the right matrix. It does this by using the row and column index information stored in the CSR and CSC formats.

For each non-zero element in the left and right matrices with matching row and column indices, it multiplies the values and adds the result to the corresponding element in the result matrix.

Finally, the method returns the result matrix.
"""
class CompressedSparseRowMatrix:
    def __init__(self, matrix):
        R, C = len(matrix), len(matrix[0])
        self.values, self.col_indices, self.row_indices = [], [], [0]
        for r in range(R):
            for c in range(C):
                if matrix[r][c] == 0: continue
                self.values.append(matrix[r][c])
                self.col_indices.append(c)
            self.row_indices.append(len(self.values))

class CompressedSparseColumnMatrix:
    def __init__(self, matrix):
        R, C = len(matrix), len(matrix[0])
        self.values, self.col_indices, self.row_indices = [], [0], []
        for c in range(C):
            for r in range(R):
                if matrix[r][c] == 0: continue
                self.values.append(matrix[r][c])
                self.row_indices.append(r)
            self.col_indices.append(len(self.values))

class Solution3:
    def multiply(self, mat1: List[List[int]], mat2: List[List[int]]) -> List[List[int]]:
        R, M, C = len(mat1), len(mat1[0]), len(mat2[0])
        left_matrix, right_matrix = CompressedSparseRowMatrix(mat1), CompressedSparseColumnMatrix(mat2)
        res = [[0]*C for _ in range(R)]
        for r, c in product(range(R), range(C)):
            left_col_ptr = left_matrix.row_indices[r]
            left_col_end = left_matrix.row_indices[r + 1]
            right_row_ptr = right_matrix.col_indices[c]
            right_row_end = right_matrix.col_indices[c + 1]
            while left_col_ptr < left_col_end and right_row_ptr < right_row_end:
                left_col_index = left_matrix.col_indices[left_col_ptr]
                right_row_index = right_matrix.row_indices[right_row_ptr]
                if left_col_index < right_row_index:
                    left_col_ptr += 1
                elif left_col_index > right_row_index:
                    right_row_ptr += 1
                else:
                    res[r][c] += left_matrix.values[left_col_ptr]*right_matrix.values[right_row_ptr]
                    left_col_ptr += 1
                    right_row_ptr += 1
        return res

In [46]:
n_rows = n_cols = 1_000
# the right matrix is much more sparse than the left matrix
mat1 = create_matrix(n_rows, n_cols, 100*n_rows).tolist()
mat2 = create_matrix(n_rows, n_cols, n_rows).tolist()

95,202
1,000,000
999
1,000,000


In [ ]:
n_rows = n_cols = 1_000
# balance sparsity
mat1 = create_matrix(n_rows, n_cols, 5*n_rows).tolist()
mat2 = create_matrix(n_rows, n_cols, 5*n_rows).tolist()

In [ ]:
"""
time is good for both implementation 2 and 3, sometimes one is better than other
"""

In [37]:
%%time
x = Solution1().multiply(mat1, mat2)

KeyboardInterrupt: 

In [47]:
%%time
x = Solution2().multiply(mat1, mat2)

CPU times: user 12.3 s, sys: 0 ns, total: 12.3 s
Wall time: 12.3 s


In [48]:
%%time
x = Solution3().multiply(mat1, mat2)

CPU times: user 5.48 s, sys: 0 ns, total: 5.48 s
Wall time: 5.51 s


# USING THE SCIPY SPARSE MATRIX LIBRARY

In [5]:
n_rows = n_cols = 1_000
# balanced sparsity
mat1 = create_matrix(n_rows, n_cols, 5*n_rows)
mat2 = create_matrix(n_rows, n_cols, 5*n_rows)


49,988
100,000,000
49,986
100,000,000


In [65]:
n_rows = n_cols = 1_000
# left leaning sparsity
mat1 = create_matrix(n_rows, n_cols, 30*n_rows)
mat2 = create_matrix(n_rows, n_cols, 2*n_rows)

29,532
1,000,000
1,998
1,000,000


In [68]:
n_rows = n_cols = 1_000
# imbalanced sparsity
mat1 = create_matrix(n_rows, n_cols, n_rows)
mat2 = create_matrix(n_rows, n_cols, 100*n_rows)

1,000
1,000,000
95,206
1,000,000


In [ ]:
"""
compressed matrices are very fast for matrix multiplication, huge speed up.
also they save thousands of memory, way lower MB of storage both in RAM and disk
"""

In [3]:
compressed_mat1 = csr_matrix(mat1)
compressed_mat2 = csc_matrix(mat2)

In [74]:
%%timeit
C = compressed_mat1.dot(compressed_mat2)

3.25 ms ± 66.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [75]:
%%timeit
C = mat1.dot(mat2)

In [8]:
compressed_mat1

<10000x10000 sparse matrix of type '<class 'numpy.int32'>'
	with 49986 stored elements in Compressed Sparse Row format>

In [18]:
scipy.sparse.save_npz('cmat1.npz', compressed_mat1)

In [19]:
scipy.sparse.load_npz('cmat1.npz')

<10000x10000 sparse matrix of type '<class 'numpy.int32'>'
	with 49986 stored elements in Compressed Sparse Row format>

In [13]:
mat1

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [17]:
np.savez('mat1.npz', x = mat1)

In [20]:
np.load('mat1.npz')['x']

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [25]:
sz1 = pathlib.Path('mat1.npz').stat().st_size
print(f"{sz1/1_024:,} MB")

390,625.25 MB


In [26]:
sz2 = pathlib.Path('cmat1.npz').stat().st_size
print(f"{sz2/1_024:,} MB")

214.7197265625 MB


In [28]:
sz1/sz2

1819.2331755149564

In [31]:
print(f"{mat1.nbytes/1_024:,} MB")

390,625.0 MB


In [32]:
print(f"{compressed_mat1.data.nbytes/1_024:,} MB")

195.2578125 MB


# DEFININING THE SPARSITY OF A MATRIX

In [40]:
def sparsity(mat):
    return 1 - np.count_nonzero(mat)/mat.size

In [41]:
sparsity(mat1)

0.99950012